In [ ]:
import cc.atlas as atlas
import cc.cartography as cartography

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use( '~/repos/clean-bold/clean-bold.mplstyle' )

# Load Data

In [ ]:
fp = '../data/example_atlas/projection.h5'
c = cartography.Cartographer.from_hdf5( fp )

# Generate Map

In [ ]:
coords, inds, pairs = c.map(
    'Hafen2019',
    max_searched = 3,
)

In [ ]:
c.publications[inds0]

In [ ]:
coords0

In [ ]:
ax, (coords0, inds0, pairs0) = c.plot_map(
    'Hafen2019',
    data = ( coords, inds, pairs ),
    max_searched = 3,
)

vor = voronoi_labels( coords0, c.publications, ax )

scipy.spatial.voronoi_plot_2d( vor, ax )

# DEBUG

In [ ]:
vdiag = skgeom.voronoi.VoronoiDiagram()

In [ ]:
for p in coords0:
    vdiag.insert( skgeom.Point2( *p ) )

for he in vdiag.edges:
    source, target = he.source(), he.target()
    if source and target:
        plt.plot([source.point().x(), target.point().x()], [source.point().y(), target.point().y()])

plt.scatter(coords0[:, 0], coords0[:, 1])

plt.axis('equal')
plt.gca().set_adjustable("box")
# plt.gca().set_xlim([-10, 10])
# plt.gca().set_ylim([-10, 10])
plt.show()

In [ ]:
import shapely

In [ ]:
import scipy

In [ ]:
points = coords0

In [ ]:
vertices = vor.vertices[region]

In [ ]:
i = 1

In [ ]:
text = ax.annotate('alskfdjal', ( 0.5, 0.5 ), )

In [ ]:
def voronoi_labels( points, labels, ax, offset_magnitude=5, annotate_kwargs={} ):

    vor = scipy.spatial.Voronoi( points )

    for i, point in enumerate( points ):
        
        # Get data for this point
        i_region = vor.point_region[i]
        region = np.array( vor.regions[i_region] )
        is_neg = region == -1
        is_on_edge = is_neg.sum() > 0
        region = region[np.invert(is_neg)]
        vertices = vor.vertices[region]
        
        # Identify closest vertix and vector pointing from it to the point
        points_all = np.concatenate( [ points[:i], points[i+1:], vertices, ], )
        v_vertices = point - points_all
        j_closest = np.argmin( np.linalg.norm( v_vertices, axis=1 ) )
        v_closest = v_vertices[j_closest]
        vhat_closest = v_closest / np.linalg.norm( v_closest )

        # Choose orientation of label
        sign = np.sign( v_closest ).astype( int )
        if sign[0] == -1:
            ha = 'right'
        else:
            ha = 'left'
        if sign[1] == -1:
            va = 'top'
        else:
            va = 'bottom'

        used_kwargs = dict(
            xycoords = 'data',
            xytext = vhat_closest * offset_magnitude,
            textcoords = 'offset points',
            ha = ha,
            va = va,
        )
        used_kwargs.update( annotate_kwargs )
        text = ax.annotate(
            text = labels[i],
            xy = point,
            **used_kwargs
        )
        
        # Assess if area is large enough
        ## Get size of text
        bbox_text = text.get_window_extent( ax.figure.canvas.get_renderer() )
        display_to_data = ax.transData.inverted()
        text_data_corners = display_to_data.transform( bbox_text.corners() )
        ax.plot( text_data_corners[:,0], text_data_corners[:,1] )
        
        ## Compare to size of voronoi area
        x, y = vertices.transpose()
        # Shoelace formula for area
        area = 0.5 * np.abs( np.dot( x, np.roll(y,1) ) - np.dot( y, np.roll(x,1) ) )

    return vor